In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


from torchvision.utils import make_grid, save_image
import torchvision.models as models
import torch.nn.functional as F
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import random
import torch
import json
import PIL


from handler import geozoom_handler
# from helpers import *
from fc_net import *
from utils import *
from sa import *

In [2]:
# random.sample(range(0, len(image_names)), 5)

In [3]:
image_names = get_png_names("../data/MEX/")[0:15]
# image_names = get_png_names("../pooling/data/MEX2/")
# image_indices = random.sample(range(0, len(image_names)), 1000)
# image_names = [image_names[i] for i in range(len(image_names)) if i in image_indices]
y = get_migrants("../../pooling/data/migration_data.json" , image_names)

15 municipalities.


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet18 = models.resnet18().to(device)
attn_model = attnNet(resnet = resnet18).to(device)
lr = .001
criterion = torch.nn.L1Loss(reduction = 'mean')
attn_optimizer = torch.optim.Adam(attn_model.parameters(), lr = lr)

butler = geozoom_handler(attn_model, 
                         device, 
                         criterion, 
                         attn_optimizer, 
                         num_thresholds = 4,
                         reduction_percent = .70,
                         convergence_dims = (224, 224),
                         perc_change_thresh = (-50, 50),
                         plot = False, 
                         v = False)

In [6]:
attn_model.sa

SelfAttention(
  (query_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (key_conv): Conv2d(256, 32, kernel_size=(1, 1), stride=(1, 1))
  (value_conv): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
  (softmax): Softmax(dim=-1)
)

In [7]:
BATCH_SIZE = 1
SPLIT = .60

train_dl, val_dl = butler.prep_attn_data(image_names, y, SPLIT, BATCH_SIZE)

In [8]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  9
Num validation:  6


In [9]:
# for i,o in train_dl:
#     print(i)
# #     print(load_inputs(i[0]).shape, o)

In [10]:
butler.train_attn_model(train_dl, val_dl)

/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Epoch:  0
  Training Loss:  2573.8209727075364
  Validation Loss:  7254.84281317393


Epoch:  1
  Training Loss:  2568.864212989807
  Validation Loss:  7331.734318415324


Epoch:  2
  Training Loss:  2565.265174335904
  Validation Loss:  7352.561503092448


Epoch:  3
  Training Loss:  2561.8527779049346
  Validation Loss:  7372.736526489258


Epoch:  4
  Training Loss:  2557.3725079430474
  Validation Loss:  9188.797526041666
Percentage above threshold:  0.5555555555555556


Epoch:  5
  Training Loss:  2554.1912981669107
  Validation Loss:  7467.206151326497
Percentage above threshold:  0.6666666666666666
  Moving to threshold:  2


Epoch:  6
  Training Loss:  2551.56677224901
  Validation Loss:  9505.354771931967


Epoch:  7
  Training Loss:  2548.5490720536973
  Validation Loss:  7410.007909138997


Epoch:  8
  Training Loss:  2544.046554353502
  Validation Loss:  8046.928695678711


Epoch:  9
  Training Loss:  2539.9669986300996
  Validation Loss:  8553.888254801432
Percentage above

In [10]:
butler.train_fc_model(train_dl, val_dl, outside_estimator = 'mlp')

MLP MAE:  4617.186772865517


/opt/conda/envs/rapids/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:585: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [12]:
image_names

['../data/MEX/484001008/pngs/484001008_2010_all_box484001008_MAY.png',
 '../data/MEX/484004004/pngs/484004004_2010_all_box484004004_MAY.png',
 '../data/MEX/484004001/pngs/484004001_2010_all_box484004001_MAY.png',
 '../data/MEX/484005010/pngs/484005010_2010_all_box484005010_MAY.png',
 '../data/MEX/484001005/pngs/484001005_2010_all_box484001005_MAY.png',
 '../data/MEX/484001007/pngs/484001007_2010_all_box484001007_MAY.png',
 '../data/MEX/484001006/pngs/484001006_2010_all_box484001006_MAY.png',
 '../data/MEX/484001001/pngs/484001001_2010_all_box484001001_MAY.png',
 '../data/MEX/484004005/pngs/484004005_2010_all_box484004005_MAY.png',
 '../data/MEX/484005001/pngs/484005001_2010_all_box484005001_MAY.png',
 '../data/MEX/484005004/pngs/484005004_2010_all_box484005004_MAY.png',
 '../data/MEX/484005008/pngs/484005008_2010_all_box484005008_MAY.png',
 '../data/MEX/484001002/pngs/484001002_2010_all_box484001002_MAY.png',
 '../data/MEX/484004006/pngs/484004006_2010_all_box484004006_MAY.png',
 '../d

In [11]:
butler.plot_attn_map(impath = '../data/MEX/484001002/pngs/484001002_2010_all_box484001002_MAY.png')


/opt/conda/envs/rapids/lib/python3.7/site-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>